In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from time import strftime,localtime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_distances

#Adding this to imports because I kept skipping the cell
scaler = StandardScaler()

In [ ]:
!pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 166 µs (started: 2021-03-22 19:19:58 +00:00)


In [ ]:
#Put this function in front of the 'chunk' cells later (or any cells you
# want to time while they are running).  You don't need to pass anything
# into it.

def st_time():
  now=datetime.now()
  st_time=now.strftime("%H:%M:%S")
  print('Start time:', st_time)
  return

# Use Cases for this Model

1) You can use the key selector for lessons because you need to practice playing in a key

2) You can us the tempo selector for practice as well for pratice playing at differnt tempos

3) Mode selector is also helpful for practice becaues it's major and minor key
4) Mode selector would help for choosing soundtracks because of mood.

5) Tempo selector may also help for choosing soundtracks for mood.

6) Speachiness and acousticness for soundstrack work.

7) Buildling your own playlists by selecting for thigns like danceability, or instrumentalness or acousticness.

8) You could also select time signature for music lessons, dance practice, or making a soundtrack.


# Weighting to keep in mind for final Model

* As a default, keep key turned off.  Keep time signature turned off.
* Play around with the tempo and maybe weight that moderatly hard in the beginning.

# Load in the Audio Feature Data Frame

In [ ]:
#This is the dataframe used from all ~82,000 songs taken from a single day's session.
# Use this data frame to put into the "df.sample(n=)"" function then add the tracer
# album file af_vv_df.

#read in all data from drive
af_tot_df = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df.csv')
#drop weird column 'Unnamed: 0' that appears
af_tot_df = af_tot_df.drop(columns=['Unnamed: 0'])
#assign track_id to a new column as 'track_id2' and set 'track_id' to index.
af_tot_df['track_idi']=af_tot_df['track_id']
#set index as 'track_id' since it's the unique identifier for every song and can call in all data.
af_tot_df.index=af_tot_df['track_idi']
#show the dataframe to see results
af_tot_df

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence,track_idi
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6tcTKTzid5MnguRqMXfmpq,6tcTKTzid5MnguRqMXfmpq,"Concerto for Strings in A Minor, RV 161: I. Al...",1bKVopvjDzU8d115PMOy3X,Vivaldi: Concertos for Strings,2QOIawHpSlOwXDvSqQ9YJR,Antonio Vivaldi,https://p.scdn.co/mp3-preview/d6535826a85d5de8...,4,https://api.spotify.com/v1/tracks/6tcTKTzid5Mn...,https://open.spotify.com/track/6tcTKTzid5MnguR...,HKI199602304,https://api.spotify.com/v1/albums/1bKVopvjDzU8...,https://open.spotify.com/album/1bKVopvjDzU8d11...,1997-03-27,day,30,album,https://i.scdn.co/image/ab67616d0000b273d21509...,https://i.scdn.co/image/ab67616d00001e02d21509...,https://i.scdn.co/image/ab67616d00004851d21509...,https://api.spotify.com/v1/artists/2QOIawHpSlO...,https://open.spotify.com/artist/2QOIawHpSlOwXD...,0.80700,0.511,0.363,0.963000,9,0.1360,-15.223,0,44,0.0339,115.397,4,98600,0.6740,6tcTKTzid5MnguRqMXfmpq
3i9Tr3Vp2gjT8EsVKGt1N3,3i9Tr3Vp2gjT8EsVKGt1N3,California - Chris Lake & Matroda Remix,1HFGTeIMHbKiEx1zZiDKWt,California,2zJ8chFLjiBHRNchfevMRI,SNBRN,https://p.scdn.co/mp3-preview/37ec14d1b31cf6fd...,2,https://api.spotify.com/v1/tracks/3i9Tr3Vp2gjT...,https://open.spotify.com/track/3i9Tr3Vp2gjT8Es...,QMLA71500357,https://api.spotify.com/v1/albums/1HFGTeIMHbKi...,https://open.spotify.com/album/1HFGTeIMHbKiEx1...,2015-05-11,day,2,album,https://i.scdn.co/image/ab67616d0000b2730e7de5...,https://i.scdn.co/image/ab67616d00001e020e7de5...,https://i.scdn.co/image/ab67616d000048510e7de5...,https://api.spotify.com/v1/artists/2zJ8chFLjiB...,https://open.spotify.com/artist/2zJ8chFLjiBHRN...,0.02260,0.783,0.810,0.257000,10,0.1190,-4.041,0,38,0.0794,124.028,4,305816,0.3800,3i9Tr3Vp2gjT8EsVKGt1N3
4Qievb8Mqy0qxdLNVl02zt,4Qievb8Mqy0qxdLNVl02zt,Been Caught Stealing,2Jkbi83HTSfqEd0CBdYwpU,Ritual De Lo Habitual,02NfyD6AlLA12crYzw5YcR,Jane's Addiction,https://p.scdn.co/mp3-preview/74af516b30ad8dd3...,5,https://api.spotify.com/v1/tracks/4Qievb8Mqy0q...,https://open.spotify.com/track/4Qievb8Mqy0qxdL...,USWB19000038,https://api.spotify.com/v1/albums/2Jkbi83HTSfq...,https://open.spotify.com/album/2Jkbi83HTSfqEd0...,1990-08-21,day,9,album,https://i.scdn.co/image/ab67616d0000b27380c825...,https://i.scdn.co/image/ab67616d00001e0280c825...,https://i.scdn.co/image/ab67616d0000485180c825...,https://api.spotify.com/v1/artists/02NfyD6AlLA...,https://open.spotify.com/artist/02NfyD6AlLA12c...,0.00326,0.639,0.929,0.050100,0,0.2420,-4.762,1,57,0.2120,103.758,4,214706,0.6880,4Qievb8Mqy0qxdLNVl02zt
3dcMbvaYRwHFXnDF9CozIs,3dcMbvaYRwHFXnDF9CozIs,Sway (Quien Sera) - 1997 - Remastered,3M4x71xrlgVrZ2SS0Jjebq,Dino: The Essential Dean Martin,49e4v89VmlDcFCMyDv9wQ9,Dean Martin,NaN,5,https://api.spotify.com/v1/tracks/3dcMbvaYRwHF...,https://open.spotify.com/track/3dcMbvaYRwHFXnD...,USCA29701333,https://api.spotify.com/v1/albums/3M4x71xrlgVr...,https://open.spotify.com/album/3M4x71xrlgVrZ2S...,2004,year,30,album,https://i.scdn.co/image/ab67616d0000b2733b126b...,https://i.scdn.co/image/ab67616d00001e023b126b...,https://i.scdn.co/image/ab67616d000048513b126b...,https://api.spotify.com/v1/artists/49e4v89VmlD...,https://open.spotify.com/artist/49e4v89VmlDcFC...,0.72700,0.611,0.288,0.000002,7,0.0792,-9.988,0,0,0.0361,120.066,4,162066,0.5830,3dcMbvaYRwHFXnDF9CozIs
4vO9dmzNRqDhFY3jD1a3P7,4vO9dmzNRqDhFY3jD1a3P7,Mississippi Queen,2tWvZmP9EVRYqmWyJPLzjP,Climbing!,7LCp4MN0SOIVWlssid9KyE,Mountain,https://p.scdn.co/mp3-preview/a94af259aa67f896...,1,https://api.spotify.com/v1/tracks/4vO9dmzNRqDh...,https://open.spotify.com/track/4vO9dmzNRqDhFY3...,U

# Subsample down for app development

Use a random sampler.

In [ ]:
#Random sampler. Set number of samples by argument "n="

def randSample(df,num):
  df=df.sample(n=num)
  return df

In [ ]:
df_rand = randSample(af_tot_df,10000)
df_rand

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence,track_idi
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5glxZ3mfbpIQseUDlyAok4,5glxZ3mfbpIQseUDlyAok4,Smack A Bitch - Dr. Fresch Remix,0JtIoiU6t0nf79euonu096,Smack A Bitch (Dr. Fresch Remix),2OaHYHb2XcFPvqL3VsyPzU,Rico Nasty,https://p.scdn.co/mp3-preview/6c8603c42a6f2a76...,1,https://api.spotify.com/v1/tracks/5glxZ3mfbpIQ...,https://open.spotify.com/track/5glxZ3mfbpIQseU...,USAT21901486,https://api.spotify.com/v1/albums/0JtIoiU6t0nf...,https://open.spotify.com/album/0JtIoiU6t0nf79e...,2019-01-25,day,1,album,https://i.scdn.co/image/ab67616d0000b273bfda6c...,https://i.scdn.co/image/ab67616d00001e02bfda6c...,https://i.scdn.co/image/ab67616d00004851bfda6c...,https://api.spotify.com/v1/artists/2OaHYHb2XcF...,https://open.spotify.com/artist/2OaHYHb2XcFPvq...,0.003450,0.722,0.887,0.000001,2,0.2980,-3.604,1,49,0.0345,93.088,4,187650,0.456,5glxZ3mfbpIQseUDlyAok4
47x1vM18oHbtCzpWujbOqM,47x1vM18oHbtCzpWujbOqM,What I Mean,0gWuQZiPI9g1RBFafXtfTn,What I Mean,1HT9k1ZSUL9IczSstOAgWJ,Roger Sanchez,https://p.scdn.co/mp3-preview/25f6ba3b06b20819...,1,https://api.spotify.com/v1/tracks/47x1vM18oHbt...,https://open.spotify.com/track/47x1vM18oHbtCzp...,GBKQU2111758,https://api.spotify.com/v1/albums/0gWuQZiPI9g1...,https://open.spotify.com/album/0gWuQZiPI9g1RBF...,2021-02-19,day,4,album,https://i.scdn.co/image/ab67616d0000b273c6de38...,https://i.scdn.co/image/ab67616d00001e02c6de38...,https://i.scdn.co/image/ab67616d00004851c6de38...,https://api.spotify.com/v1/artists/1HT9k1ZSUL9...,https://open.spotify.com/artist/1HT9k1ZSUL9Icz...,0.060600,0.840,0.973,0.034300,9,0.0825,-4.858,0,43,0.1200,129.964,4,180000,0.895,47x1vM18oHbtCzpWujbOqM
0IS8n5zMLsZxbTPjghaqgW,0IS8n5zMLsZxbTPjghaqgW,Don't Let Them - Flyboy Remix,3oG5zTsAHoHMZCAtlJwAVQ,Don't Let Them (Remixes),4LKB1IkCINDDjEX8iS7glI,Young Bombs,https://p.scdn.co/mp3-preview/04467a3404c8dc11...,2,https://api.spotify.com/v1/tracks/0IS8n5zMLsZx...,https://open.spotify.com/track/0IS8n5zMLsZxbTP...,USUG11902415,https://api.spotify.com/v1/albums/3oG5zTsAHoHM...,https://open.spotify.com/album/3oG5zTsAHoHMZCA...,2019-07-19,day,4,album,https://i.scdn.co/image/ab67616d0000b2737e9cb1...,https://i.scdn.co/image/ab67616d00001e027e9cb1...,https://i.scdn.co/image/ab67616d000048517e9cb1...,https://api.spotify.com/v1/artists/4LKB1IkCIND...,https://open.spotify.com/artist/4LKB1IkCINDDjE...,0.026500,0.485,0.787,0.000116,5,0.1070,-5.160,1,52,0.1900,103.004,4,228349,0.330,0IS8n5zMLsZxbTPjghaqgW
6ssujYDvWvyC5H8ZBjRW3K,6ssujYDvWvyC5H8ZBjRW3K,Total War,1cvZ2jWkoHj5uxdl3P8VP1,Neutralize the Threat,4ITkzAQWPILwWpEWJxHB9g,Earth Crisis,https://p.scdn.co/mp3-preview/1e47203b6dbaf4ec...,3,https://api.spotify.com/v1/tracks/6ssujYDvWvyC...,https://open.spotify.com/track/6ssujYDvWvyC5H8...,US4E41179003,https://api.spotify.com/v1/albums/1cvZ2jWkoHj5...,https://open.spotify.com/album/1cvZ2jWkoHj5uxd...,2011,year,10,album,https://i.scdn.co/image/ab67616d0000b273cc844e...,https://i.scdn.co/image/ab67616d00001e02cc844e...,https://i.scdn.co/image/ab67616d00004851cc844e...,https://api.spotify.com/v1/artists/4ITkzAQWPIL...,https://open.spotify.com/artist/4ITkzAQWPILwWp...,0.000680,0.546,0.984,0.085800,0,0.0837,-3.359,1,37,0.0889,131.999,4,218773,0.251,6ssujYDvWvyC5H8ZBjRW3K
724HbPYQXUE0Ad9iDnnLHO,724HbPYQXUE0Ad9iDnnLHO,Delta Lady,1d0MHN80sQmt83jIkAr94F,Classic Joe Cocker - The Universal Masters Col...,3pFCERyEiP5xeN2EsPXhjI,Joe Cocker,NaN,5,https://api.spotify.com/v1/tracks/724HbPYQXUE0...,https://open.spotify.com/track/724HbPYQXUE0Ad9...,USAM16900386,https://api.spotify.com/v1

## Concatenate random subsample with tracer album

### Import tracer data and format to match main data

In [ ]:
#Tracer album to be put into library after random sampling.
#read in all data from drive
af_vv_df = pd.read_csv('/content/drive/MyDrive/Project5/af_vv_df.csv')
##drop weird column 'Unnamed: 0' that appears 
af_vv_df = af_vv_df.drop(columns=['Unnamed: 0'])
#assign track_id to a new column as 'track_id2' and set 'track_id' to index.
af_vv_df['track_idi']=af_vv_df['track_id']
#set index as 'track_id' since it's the unique identifier for every song and can call in all data.
af_vv_df.index=af_vv_df['track_idi']
#show the dataframe to see results
af_vv_df

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence,track_idi
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57lYOdic1C2GVyTPZsJhlB,57lYOdic1C2GVyTPZsJhlB,Necksweat,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/dee897818c302af8...,1,https://api.spotify.com/v1/tracks/57lYOdic1C2G...,https://open.spotify.com/track/57lYOdic1C2GVyT...,TCADA1792090,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.749000,0.352,0.369,0.929,9,0.1080,-9.028,1,0,0.0445,123.566,4,300176,0.3590,57lYOdic1C2GVyTPZsJhlB
3Rv7s6zTx6KTYZfrpBdW4C,3Rv7s6zTx6KTYZfrpBdW4C,Soar,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/2f8fe8e08ca3cdae...,2,https://api.spotify.com/v1/tracks/3Rv7s6zTx6KT...,https://open.spotify.com/track/3Rv7s6zTx6KTYZf...,TCADA1792174,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.719000,0.349,0.367,0.423,7,0.0568,-7.647,1,0,0.0430,158.815,4,246319,0.1330,3Rv7s6zTx6KTYZfrpBdW4C
0Qj2eDY1Nku7OGymMs8HGz,0Qj2eDY1Nku7OGymMs8HGz,Cascade,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/6f095534525eec32...,3,https://api.spotify.com/v1/tracks/0Qj2eDY1Nku7...,https://open.spotify.com/track/0Qj2eDY1Nku7OGy...,TCADA1792176,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.913000,0.496,0.167,0.964,1,0.0853,-8.508,0,0,0.0445,99.872,4,253442,0.2380,0Qj2eDY1Nku7OGymMs8HGz
2pgLRrQEvDwd6jTXGDswQm,2pgLRrQEvDwd6jTXGDswQm,Ausländer,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/f6ec6a950b478ce3...,4,https://api.spotify.com/v1/tracks/2pgLRrQEvDwd...,https://open.spotify.com/track/2pgLRrQEvDwd6jT...,TCADA1792177,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/image/ab67616d0000b273368290...,https://i.scdn.co/image/ab67616d00001e02368290...,https://i.scdn.co/image/ab67616d00004851368290...,https://api.spotify.com/v1/artists/3hFXoighTNp...,https://open.spotify.com/artist/3hFXoighTNpdku...,0.655000,0.318,0.272,0.962,4,0.1050,-11.335,0,0,0.0339,97.206,4,361024,0.0389,2pgLRrQEvDwd6jTXGDswQm
5SzbfdSfxmmlyTEZcpb3EH,5SzbfdSfxmmlyTEZcpb3EH,County Fair,72yXt1N6iw8TnCG4KeX5pr,Immolate,3hFXoighTNpdkuXcwFu09n,Vaso Vagal,https://p.scdn.co/mp3-preview/f148a09e067891d4...,5,https://api.spotify.com/v1/tracks/5SzbfdSfxmml...,https://open.spotify.com/track/5SzbfdSfxmmlyTE...,TCADA1792179,https://api.spotify.com/v1/albums/72yXt1N6iw8T...,https://open.spotify.com/album/72yXt1N6iw8TnCG...,2017-05-12,day,9,album,https://i.scdn.co/i

### Concate main data and tracer *only once*

Otherwise you'll keep adding copies of the tracer album


In [ ]:
#concat the main data 'af_rand_df' plus tracer album 'af_vv_df'
af_rand_10_df = df_rand
af_rand_10_df = pd.concat((af_rand_10_df,af_vv_df),axis=0)
af_rand_10_df

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence,track_idi
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5glxZ3mfbpIQseUDlyAok4,5glxZ3mfbpIQseUDlyAok4,Smack A Bitch - Dr. Fresch Remix,0JtIoiU6t0nf79euonu096,Smack A Bitch (Dr. Fresch Remix),2OaHYHb2XcFPvqL3VsyPzU,Rico Nasty,https://p.scdn.co/mp3-preview/6c8603c42a6f2a76...,1,https://api.spotify.com/v1/tracks/5glxZ3mfbpIQ...,https://open.spotify.com/track/5glxZ3mfbpIQseU...,USAT21901486,https://api.spotify.com/v1/albums/0JtIoiU6t0nf...,https://open.spotify.com/album/0JtIoiU6t0nf79e...,2019-01-25,day,1,album,https://i.scdn.co/image/ab67616d0000b273bfda6c...,https://i.scdn.co/image/ab67616d00001e02bfda6c...,https://i.scdn.co/image/ab67616d00004851bfda6c...,https://api.spotify.com/v1/artists/2OaHYHb2XcF...,https://open.spotify.com/artist/2OaHYHb2XcFPvq...,0.003450,0.722,0.887,0.000001,2,0.2980,-3.604,1,49,0.0345,93.088,4,187650,0.456,5glxZ3mfbpIQseUDlyAok4
47x1vM18oHbtCzpWujbOqM,47x1vM18oHbtCzpWujbOqM,What I Mean,0gWuQZiPI9g1RBFafXtfTn,What I Mean,1HT9k1ZSUL9IczSstOAgWJ,Roger Sanchez,https://p.scdn.co/mp3-preview/25f6ba3b06b20819...,1,https://api.spotify.com/v1/tracks/47x1vM18oHbt...,https://open.spotify.com/track/47x1vM18oHbtCzp...,GBKQU2111758,https://api.spotify.com/v1/albums/0gWuQZiPI9g1...,https://open.spotify.com/album/0gWuQZiPI9g1RBF...,2021-02-19,day,4,album,https://i.scdn.co/image/ab67616d0000b273c6de38...,https://i.scdn.co/image/ab67616d00001e02c6de38...,https://i.scdn.co/image/ab67616d00004851c6de38...,https://api.spotify.com/v1/artists/1HT9k1ZSUL9...,https://open.spotify.com/artist/1HT9k1ZSUL9Icz...,0.060600,0.840,0.973,0.034300,9,0.0825,-4.858,0,43,0.1200,129.964,4,180000,0.895,47x1vM18oHbtCzpWujbOqM
0IS8n5zMLsZxbTPjghaqgW,0IS8n5zMLsZxbTPjghaqgW,Don't Let Them - Flyboy Remix,3oG5zTsAHoHMZCAtlJwAVQ,Don't Let Them (Remixes),4LKB1IkCINDDjEX8iS7glI,Young Bombs,https://p.scdn.co/mp3-preview/04467a3404c8dc11...,2,https://api.spotify.com/v1/tracks/0IS8n5zMLsZx...,https://open.spotify.com/track/0IS8n5zMLsZxbTP...,USUG11902415,https://api.spotify.com/v1/albums/3oG5zTsAHoHM...,https://open.spotify.com/album/3oG5zTsAHoHMZCA...,2019-07-19,day,4,album,https://i.scdn.co/image/ab67616d0000b2737e9cb1...,https://i.scdn.co/image/ab67616d00001e027e9cb1...,https://i.scdn.co/image/ab67616d000048517e9cb1...,https://api.spotify.com/v1/artists/4LKB1IkCIND...,https://open.spotify.com/artist/4LKB1IkCINDDjE...,0.026500,0.485,0.787,0.000116,5,0.1070,-5.160,1,52,0.1900,103.004,4,228349,0.330,0IS8n5zMLsZxbTPjghaqgW
6ssujYDvWvyC5H8ZBjRW3K,6ssujYDvWvyC5H8ZBjRW3K,Total War,1cvZ2jWkoHj5uxdl3P8VP1,Neutralize the Threat,4ITkzAQWPILwWpEWJxHB9g,Earth Crisis,https://p.scdn.co/mp3-preview/1e47203b6dbaf4ec...,3,https://api.spotify.com/v1/tracks/6ssujYDvWvyC...,https://open.spotify.com/track/6ssujYDvWvyC5H8...,US4E41179003,https://api.spotify.com/v1/albums/1cvZ2jWkoHj5...,https://open.spotify.com/album/1cvZ2jWkoHj5uxd...,2011,year,10,album,https://i.scdn.co/image/ab67616d0000b273cc844e...,https://i.scdn.co/image/ab67616d00001e02cc844e...,https://i.scdn.co/image/ab67616d00004851cc844e...,https://api.spotify.com/v1/artists/4ITkzAQWPIL...,https://open.spotify.com/artist/4ITkzAQWPILwWp...,0.000680,0.546,0.984,0.085800,0,0.0837,-3.359,1,37,0.0889,131.999,4,218773,0.251,6ssujYDvWvyC5H8ZBjRW3K
724HbPYQXUE0Ad9iDnnLHO,724HbPYQXUE0Ad9iDnnLHO,Delta Lady,1d0MHN80sQmt83jIkAr94F,Classic Joe Cocker - The Universal Masters Col...,3pFCERyEiP5xeN2EsPXhjI,Joe Cocker,NaN,5,https://api.spotify.com/v1/tracks/724HbPYQXUE0...,https://open.spotify.com/track/724HbPYQXUE0Ad9...,USAM16900386,https://api.spotify.com/v1

### Format the concatenated the final dataframe

In [ ]:
#assign track_id to a new column as 'track_id2' and set 'track_id' to index.
af_rand_10_df['track_idi']=af_rand_10_df['track_id']
#set index as 'track_id' since it's the unique identifier for every song and can call in all data.
af_rand_10_df.index=af_rand_10_df['track_idi']
#drop 'track_idi'
af_rand_10_df = af_rand_10_df.drop(columns=['track_idi'])
#Check the dataframe
af_rand_10_df

,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5glxZ3mfbpIQseUDlyAok4,5glxZ3mfbpIQseUDlyAok4,Smack A Bitch - Dr. Fresch Remix,0JtIoiU6t0nf79euonu096,Smack A Bitch (Dr. Fresch Remix),2OaHYHb2XcFPvqL3VsyPzU,Rico Nasty,https://p.scdn.co/mp3-preview/6c8603c42a6f2a76...,1,https://api.spotify.com/v1/tracks/5glxZ3mfbpIQ...,https://open.spotify.com/track/5glxZ3mfbpIQseU...,USAT21901486,https://api.spotify.com/v1/albums/0JtIoiU6t0nf...,https://open.spotify.com/album/0JtIoiU6t0nf79e...,2019-01-25,day,1,album,https://i.scdn.co/image/ab67616d0000b273bfda6c...,https://i.scdn.co/image/ab67616d00001e02bfda6c...,https://i.scdn.co/image/ab67616d00004851bfda6c...,https://api.spotify.com/v1/artists/2OaHYHb2XcF...,https://open.spotify.com/artist/2OaHYHb2XcFPvq...,0.003450,0.722,0.887,0.000001,2,0.2980,-3.604,1,49,0.0345,93.088,4,187650,0.456
47x1vM18oHbtCzpWujbOqM,47x1vM18oHbtCzpWujbOqM,What I Mean,0gWuQZiPI9g1RBFafXtfTn,What I Mean,1HT9k1ZSUL9IczSstOAgWJ,Roger Sanchez,https://p.scdn.co/mp3-preview/25f6ba3b06b20819...,1,https://api.spotify.com/v1/tracks/47x1vM18oHbt...,https://open.spotify.com/track/47x1vM18oHbtCzp...,GBKQU2111758,https://api.spotify.com/v1/albums/0gWuQZiPI9g1...,https://open.spotify.com/album/0gWuQZiPI9g1RBF...,2021-02-19,day,4,album,https://i.scdn.co/image/ab67616d0000b273c6de38...,https://i.scdn.co/image/ab67616d00001e02c6de38...,https://i.scdn.co/image/ab67616d00004851c6de38...,https://api.spotify.com/v1/artists/1HT9k1ZSUL9...,https://open.spotify.com/artist/1HT9k1ZSUL9Icz...,0.060600,0.840,0.973,0.034300,9,0.0825,-4.858,0,43,0.1200,129.964,4,180000,0.895
0IS8n5zMLsZxbTPjghaqgW,0IS8n5zMLsZxbTPjghaqgW,Don't Let Them - Flyboy Remix,3oG5zTsAHoHMZCAtlJwAVQ,Don't Let Them (Remixes),4LKB1IkCINDDjEX8iS7glI,Young Bombs,https://p.scdn.co/mp3-preview/04467a3404c8dc11...,2,https://api.spotify.com/v1/tracks/0IS8n5zMLsZx...,https://open.spotify.com/track/0IS8n5zMLsZxbTP...,USUG11902415,https://api.spotify.com/v1/albums/3oG5zTsAHoHM...,https://open.spotify.com/album/3oG5zTsAHoHMZCA...,2019-07-19,day,4,album,https://i.scdn.co/image/ab67616d0000b2737e9cb1...,https://i.scdn.co/image/ab67616d00001e027e9cb1...,https://i.scdn.co/image/ab67616d000048517e9cb1...,https://api.spotify.com/v1/artists/4LKB1IkCIND...,https://open.spotify.com/artist/4LKB1IkCINDDjE...,0.026500,0.485,0.787,0.000116,5,0.1070,-5.160,1,52,0.1900,103.004,4,228349,0.330
6ssujYDvWvyC5H8ZBjRW3K,6ssujYDvWvyC5H8ZBjRW3K,Total War,1cvZ2jWkoHj5uxdl3P8VP1,Neutralize the Threat,4ITkzAQWPILwWpEWJxHB9g,Earth Crisis,https://p.scdn.co/mp3-preview/1e47203b6dbaf4ec...,3,https://api.spotify.com/v1/tracks/6ssujYDvWvyC...,https://open.spotify.com/track/6ssujYDvWvyC5H8...,US4E41179003,https://api.spotify.com/v1/albums/1cvZ2jWkoHj5...,https://open.spotify.com/album/1cvZ2jWkoHj5uxd...,2011,year,10,album,https://i.scdn.co/image/ab67616d0000b273cc844e...,https://i.scdn.co/image/ab67616d00001e02cc844e...,https://i.scdn.co/image/ab67616d00004851cc844e...,https://api.spotify.com/v1/artists/4ITkzAQWPIL...,https://open.spotify.com/artist/4ITkzAQWPILwWp...,0.000680,0.546,0.984,0.085800,0,0.0837,-3.359,1,37,0.0889,131.999,4,218773,0.251
724HbPYQXUE0Ad9iDnnLHO,724HbPYQXUE0Ad9iDnnLHO,Delta Lady,1d0MHN80sQmt83jIkAr94F,Classic Joe Cocker - The Universal Masters Col...,3pFCERyEiP5xeN2EsPXhjI,Joe Cocker,NaN,5,https://api.spotify.com/v1/tracks/724HbPYQXUE0...,https://open.spotify.com/track/724HbPYQXUE0Ad9...,USAM16900386,https://api.spotify.com/v1/albums/1d0MHN80sQmt...,https://open.spotify.com/album/1d0MHN80sQmt83j...,2000-01-07,day,15,album,https

# Cosine distance matrix


In [ ]:
#Testing times for local Jupyter Notebook are:
#Tested with all 14 audio features and 10000/80000 rows.  Execution time was 1.6s = 1.6s
#                                      20000/80000 rows.  Exuection time was 9.64s = 9.64s
#                                      30000/80000 rows.  Exeuction time was 3m9s = 189s
#                                      40000/80000 rows.  ESTIMATED time is 19-63m = 1,140-3,780s
#                                      50000/80000 rows.  ESTIMATED time is 114-1260m = 6,840-75600s. (1.2-21hours)

#Testing times with Google Colab are:
#Tested with all 14 audio features and 10000/80000 rows.  Execution time was 1.1s = 1.1s
#                                      20000/80000 rows.  Exuection time was 4.0 = 4.0s
#                                      30000/80000 rows.  Exeuction time was 10.8s = 10.8s
#                          *High-RAM   40000/80000 rows.  Execution time is 8.8s = 8.8s
#                          *High-RAM   50000/80000 rows.  ESTIMATED time is 114-1260m = 6,840-75600s. (1.2-21hours)
#
# GPU and TPU slow the operation down by about 1.5-2X

def cos_dis(df):
  st_time()
  df_scaled_trans = scaler.fit_transform(df.iloc[:,22:]) #skips first 23 columns, which are metadata.
  df_cos = cosine_distances(df_scaled_trans)
  df_cos = pd.DataFrame(df_cos)
  return df_cos

In [ ]:
#Run the cosine distance calculation.
#Assign the index as 'track_id' from the random matrix.  This will allow you to always
# have a reference dataframe to fall back on.
#40000 rows took 20.1s for HIGH-RAM

cos_df = cos_dis(af_rand_10_df)
cos_df

Start time: 19:24:39


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000,10001,10002,10003,10004,10005,10006,10007,10008
0,0.000000,0.656939,0.328480,0.319992,0.639642,0.254998,0.697075,1.230263,1.266982,1.338844,0.866907,0.426462,0.902879,1.146615,0.581533,0.939164,1.245559,1.251561,0.575584,1.368068,0.517896,0.913716,0.942722,0.975855,0.822149,1.361676,0.496007,0.300921,0.385518,1.021090,0.861700,0.775854,0.936120,1.396462,1.122386,0.678440,0.825679,1.483297,0.520551,0.768590,...,1.470733,1.072499,1.520165,0.224445,0.592265,1.290646,0.189138,0.623524,1.661667,1.256085,0.683262,1.250947,1.027154,0.705256,1.058455,0.414425,0.659970,1.343134,0.461336,1.343032,1.687896,1.501147,1.406497,0.370263,1.002884,1.555554,0.593108,1.062619,0.520303,1.397674,1.107986,1.558289,1.543470,1.433623,1.520627,1.507800,0.676800,1.475938,1.364785,1.112277
1,0.656939,0.000000,0.788883,0.857266,0.955928,1.042175,0.465130,1.422207,1.609460,1.611364,1.098544,1.115168,1.393716,0.573914,0.306655,0.624561,1.562138,1.604111,0.339990,1.281596,1.067226,1.489432,0.415114,0.980145,1.313031,1.476355,0.258366,0.502314,0.338669,1.179115,0.245681,0.715934,1.285212,1.036118,1.386648,1.058534,0.327583,1.256000,0.540111,0.556480,...,1.257539,1.188496,1.682972,0.820308,0.751557,1.116036,0.530544,0.623373,1.526524,1.735395,0.313551,1.738462,1.007673,0.670751,0.908326,0.584789,0.799644,1.575788,0.827607,1.178195,1.735085,1.718512,1.166256,0.675198,1.473050,1.665683,0.519221,0.478557,0.754043,1.608619,0.765528,1.377439,1.464840,1.361880,1.465658,1.171263,1.256844,1.515658,1.421757,1.432370
2,0.328480,0.788883,0.000000,0.268472,1.054504,0.428394,0.757735,1.535392,1.111631,1.338627,1.214789,0.389134,0.681534,1.310390,0.440697,0.859552,1.077269,1.141904,0.787184,1.252245,0.747842,1.103930,0.692412,1.025880,0.733844,1.579984,0.680376,0.384801,0.717056,0.874791,0.720104,1.158656,1.020420,1.288544,1.016545,0.696363,0.724245,1.404656,0.335750,0.904139,...,1.369641,0.964813,1.205670,0.420795,0.257842,1.271027,0.384475,0.555605,1.594673,1.121385,0.597422,1.271665,0.817189,0.358318,0.882521,0.536045,0.673928,1.209025,0.406122,1.310374,1.360016,1.408400,1.290875,0.507571,1.161102,1.341075,1.045551,1.193537,0.547940,1.530016,1.021430,1.347754,1.203580,1.531586,1.400610,1.407848,0.873249,1.221611,1.123074,1.405028
3,0.319992,0.857266,0.268472,0.000000,0.682909,0.180834,0.832357,1.424665,1.036652,1.269194,0.940369,0.564196,0.440170,1.274276,0.722647,1.120577,1.098138,1.065539,0.795297,1.319323,0.444088,0.804901,0.731296,0.725744,0.903628,1.465894,0.858317,0.427087,0.505279,0.887487,0.900277,0.982332,0.591259,1.314990,1.029345,0.432729,1.029108,1.510575,0.572993,0.581593,...,1.429903,0.823151,1.419342,0.479408,0.779563,1.216005,0.260746,0.907455,1.533812,1.090601,0.687455,1.112516,1.340383,0.586726,1.196359,0.380538,0.817907,1.413583,0.304016,1.419048,1.440957,1.180142,1.187457,0.385587,1.090909,1.152382,0.853364,1.083698,0.588094,1.195991,0.989977,1.297031,0.992374,1.215761,1.215658,1.118254,0.652935,1.241015,1.045541,1.172009
4,0.639642,0.955928,1.054504,0.682909,0.000000,0.618749,0.536504,1.097565,1.216093,0.845487,0.680774,0.927768,0.795089,1.317914,1.355016,1.304298,1.245996,1.214332,0.548159,1.076372,0.503365,0.449253,1.295225,0.636541,1.044976,0.772435,0.864185,0.570926,0.393495,0.909825,0.931661,0.968968,0.453892,1.302342,1.235410,0.937570,1.484482,1.223191,1.229663,0.719369,...,1.498962,0.901653,1.351797,0.726509,1.376723,1.081268,0.450953,1.140656,1.426165,1.293203,0.853707,0.898638,1.457646,1.282736,1.335296,0.436488,0.846840,1.508353,0.950233,0.928419,1.448814,1.244692,1.390013,0.440497,0.900154,1.184378,0.591125,1.199146,0.939134,0.813298,1.159728,1.137113,1.054410,1.116743,1.306846,0.746839,0.665904,1.288797,1.097652,0.964681
...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
#Make sure that the 'df.values' was used because when we tried 'data=df' the RAM crashed even when it was just 100x100 rows x columns).
#Note: This adds the 'df.values' on top of what would be all NaN's.
#Setting the index for 'cos_df' as the index from the input dataframe prior to calculating cosine distance.
# Notice that the index is 'track_idi' and the columns are from 'track_id'

cos_df = pd.DataFrame(cos_df.values,columns=af_rand_10_df['track_id'],index=af_rand_10_df.index)
cos_df

track_id,5glxZ3mfbpIQseUDlyAok4,47x1vM18oHbtCzpWujbOqM,0IS8n5zMLsZxbTPjghaqgW,6ssujYDvWvyC5H8ZBjRW3K,724HbPYQXUE0Ad9iDnnLHO,2On5KqXqvJJKF9e2TwzRil,2P42mn8y5OCL8iBD8n3s8e,5FXIXX9FniNJfHd9DJrOop,7FsM2MPjPCdgYld4NsvO0f,5K2lMPE2hF7JPtlQapg5Ej,2MLsWnBmhbkyypQTJ7DFgw,4M0qjAImpGvUaGE2eiA7Cq,5fpu3LhQlnuc60sjDbssjW,67gzqYKrhQZam3TtAeK4K7,1KixkQVDUHggZMU9dUobgm,2imAA5ofcn4dCRVNjKyEq3,49jnjQA2w2gP7KTj9HzLaN,7EWwiyBXMaLzoSXwWZkYyU,35kyKr5ArvTnFlldwR7JAD,7DcOxkCEGXgieZAE60Ksbe,5WLdHm0xE5GRnQeaqstURR,6RAyxEP4b5HqGBApibuYIr,7g2zJC8JmpZF2vcXjZOHOe,0FMkEykWCe2vpzv6H3Dsx5,5IZU5UmMN4Q1up8pnW2hWL,0AWl5zGTaiwcnF0MDOuhOP,5m578MgxuEC956dulprCe8,2I6Jfs4VsWwpo3SI0yMEVz,6SrvyfmAxkLt9CULhf9a49,0l1BpftQspErgqlSzzmIEm,2cY8x0jQ1wWEipUCouclsD,0MIq5zqSNk6XH4BoinuakO,3SEZgDncfa1sqjThdI9JcJ,1XvMG16zHYfz9Dr6vSZYVf,69dDr3w8IIcOGKwlW5XntT,5Gdgx8sxajwAUykuwha8dj,1gdvQRgBMwyofkeCH6hi4o,6g2TQT6xWLXHQ9sDW7m5xS,3CYH422oy1cZNoo0GTG1TK,3kZaVSg3uW3tpDuHVwhgqo,...,2KFLWIbES2IaA2WwJ9jmhT,58lrpY84WlW7a3aD2YJYsV,5xwlLnghuHyBoPl0UOCF4K,1FlIzyCvRFQUE4pc7PiaIo,5pTpq4JWJHkZtcO2BjBN4V,2srLwuADGzJdpszWbeZsgh,77S5QQdlLJmr3HT8gMuWS9,63clD8ke57qeOw7czBPvoz,7HtiS6ubdQ8vUOMDLltiT3,4x9nVGF0fDaffc1LkBmgNP,05cjQouqyDYG6F2NSNJL7E,4F76PwZIzy2gW0wcGQuzN6,6iewIywVK4V0SQfSCZonCR,5mQlRdP91RtczegYXG3X63,4UoYb2HJklHzXbVA5klJGt,17KR2xwfXqLDwfibhi7zqZ,59OZiGqFfWmry6o9pTx8i8,3WiissyVFRpCVrqbyPKZxZ,4BYGxv4rxSNcTgT3DsFB9o,3jgZ09cCEn3udRr7KV86oJ,2S3qN8s6dS8QR5PthnqI2P,2jEfrzuEVCkNLbCe3EI5zY,7HT6TuyQsBNGghhYYzInsU,2XwYeBzumtjxVbFWwhJpsy,6Ktr87bevDceLjUC3xtLAt,6G9CeRGryVV0ZX60fkMEeO,6TDGueJuplnLpbww2YQPM0,7HAliS4AAbY8kWnhotOyTx,2Ajr7w89EnxLZ6KONUAguF,3o5ibU6l0GRUzzg8Hlk6xJ,7lMjqRbsChqroWRRkhw3ge,57lYOdic1C2GVyTPZsJhlB,3Rv7s6zTx6KTYZfrpBdW4C,0Qj2eDY1Nku7OGymMs8HGz,2pgLRrQEvDwd6jTXGDswQm,5SzbfdSfxmmlyTEZcpb3EH,6wUEUO6bE1R84LAB7TIAIs,09u4feGdXwYsvH7hWObLPw,0q4ADFP0VAFpTKyAnWuU0e,7mxdqQjhmmv1iPdrE6VEmB
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5glxZ3mfbpIQseUDlyAok4,0.000000,0.656939,0.328480,0.319992,0.639642,0.254998,0.697075,1.230263,1.266982,1.338844,0.866907,0.426462,0.902879,1.146615,0.581533,0.939164,1.245559,1.251561,0.575584,1.368068,0.517896,0.913716,0.942722,0.975855,0.822149,1.361676,0.496007,0.300921,0.385518,1.021090,0.861700,0.775854,0.936120,1.396462,1.122386,0.678440,0.825679,1.483297,0.520551,0.768590,...,1.470733,1.072499,1.520165,0.224445,0.592265,1.290646,0.189138,0.623524,1.661667,1.256085,0.683262,1.250947,1.027154,0.705256,1.058455,0.414425,0.659970,1.343134,0.461336,1.343032,1.687896,1.501147,1.406497,0.370263,1.002884,1.555554,0.593108,1.062619,0.520303,1.397674,1.107986,1.558289,1.543470,1.433623,1.520627,1.507800,0.676800,1.475938,1.364785,1.112277
47x1vM18oHbtCzpWujbOqM,0.656939,0.000000,0.788883,0.857266,0.955928,1.042175,0.465130,1.422207,1.609460,1.611364,1.098544,1.115168,1.393716,0.573914,0.306655,0.624561,1.562138,1.604111,0.339990,1.281596,1.067226,1.489432,0.415114,0.980145,1.313031,1.476355,0.258366,0.502314,0.338669,1.179115,0.245681,0.715934,1.285212,1.036118,1.386648,1.058534,0.327583,1.256000,0.540111,0.556480,...,1.257539,1.188496,1.682972,0.820308,0.751557,1.116036,0.530544,0.623373,1.526524,1.735395,0.313551,1.738462,1.007673,0.670751,0.908326,0.584789,0.799644,1.575788,0.827607,1.178195,1.735085,1.718512,1.166256,0.675198,1.473050,1.665683,0.519221,0.478557,0.754043,1.608619,0.765528,1.377439,1.464840,1.361880,1.465658,1.171263,1.256844,1.515658,1.421757,1.432370
0IS8n5zMLsZxbTPjghaqgW,0.328480,0.788883,0.000000,0.268472,1.054504,0.428394,0.757735,1.535392,1.111631,1.338627,1.214789,0.389134,0.681534,1.310390,0.440697,0.859552,1.077269,1.141904,0.787184,1.252245,0.747842,1.103930,0.692412,1.025880,0.733844,1.579984,0.680376,0.384801,0.717056,0.874791,0.720104,1.158656,1.020420,1.288544,1.016545,0.696363,0.724245,1.404656,0.335750,0.904139,...,1.369641,0.964813,1.205670,0.420795,0.257842,1.271027,0.384475,0.555605,1.594673,1.121385,0.597422,1.271665,0.817189,0.358318,0.882521,0.536045,0.673928,1.209025,0.406122,1.31

# Single column cosine distance

# Filter and Sort

In [ ]:
#Execution notes for Jupyter Lab/Notebook:
#Time exuection time was 49s for 20000 songs with no filtering.
# Adding the filtering of values <0.2 brought it down to 0.28s.
# This is 245 times faster...so I'd recommend doing it.

#Also, the filter here will help yoru RAM from from overflowing
# and crashing your session.  The computer this was written on
# has 8GB of RAM.

#The structure of this line of code is:
# filtered_df = previous_df[previous_df.Track <operator like ">,<,="> value]
# noting that Tracks are column names.

#Execution notes for Google Colab:
# Make sure to use the syntax formatting for the column seen below to guarantee
# the boolean will work.

def filterSort(track_id,filter_value,head_size):
  #This sort won't work on the new single column for some reason.  So i'm having to refernece the main dataframe
  cos_df_fil = cos_df[cos_df[track_id]<filter_value]
  # #move 'track_id2' to column=0 by doing a list comprehension where the dataframe = the column you want to move plus every column that isn't equal to that column.
  cos_df_fil = cos_df_fil[[track_id] + [col for col in cos_df_fil.columns if col != track_id]]
  cos_df_fil = cos_df_fil.sort_values(by=[track_id], ascending=True)
  #Put in statement where album_id or arist_id cannot be the same as the input.
  return cos_df_fil.head(head_size)



*   List item
*   List item



In [ ]:
#Run filterSort to get filtered results for a given track.
#'track_id' = track_id, 'filter_value = limit on filter results.  Returned results
# will only be LESS THAN this value. 'head_size' = the number of results shows
# when the program is executed.

cos_df_fil = filterSort('57lYOdic1C2GVyTPZsJhlB',0.2,10)
cos_df_fil

track_id,57lYOdic1C2GVyTPZsJhlB,5glxZ3mfbpIQseUDlyAok4,47x1vM18oHbtCzpWujbOqM,0IS8n5zMLsZxbTPjghaqgW,6ssujYDvWvyC5H8ZBjRW3K,724HbPYQXUE0Ad9iDnnLHO,2On5KqXqvJJKF9e2TwzRil,2P42mn8y5OCL8iBD8n3s8e,2P42mn8y5OCL8iBD8n3s8e,5FXIXX9FniNJfHd9DJrOop,7FsM2MPjPCdgYld4NsvO0f,5K2lMPE2hF7JPtlQapg5Ej,2MLsWnBmhbkyypQTJ7DFgw,4M0qjAImpGvUaGE2eiA7Cq,5fpu3LhQlnuc60sjDbssjW,67gzqYKrhQZam3TtAeK4K7,1KixkQVDUHggZMU9dUobgm,2imAA5ofcn4dCRVNjKyEq3,49jnjQA2w2gP7KTj9HzLaN,7EWwiyBXMaLzoSXwWZkYyU,35kyKr5ArvTnFlldwR7JAD,7DcOxkCEGXgieZAE60Ksbe,5WLdHm0xE5GRnQeaqstURR,6RAyxEP4b5HqGBApibuYIr,7g2zJC8JmpZF2vcXjZOHOe,0FMkEykWCe2vpzv6H3Dsx5,5IZU5UmMN4Q1up8pnW2hWL,5IZU5UmMN4Q1up8pnW2hWL,0AWl5zGTaiwcnF0MDOuhOP,5m578MgxuEC956dulprCe8,2I6Jfs4VsWwpo3SI0yMEVz,6SrvyfmAxkLt9CULhf9a49,0l1BpftQspErgqlSzzmIEm,2cY8x0jQ1wWEipUCouclsD,0MIq5zqSNk6XH4BoinuakO,3SEZgDncfa1sqjThdI9JcJ,1XvMG16zHYfz9Dr6vSZYVf,69dDr3w8IIcOGKwlW5XntT,5Gdgx8sxajwAUykuwha8dj,1gdvQRgBMwyofkeCH6hi4o,...,58lrpY84WlW7a3aD2YJYsV,5xwlLnghuHyBoPl0UOCF4K,1FlIzyCvRFQUE4pc7PiaIo,5pTpq4JWJHkZtcO2BjBN4V,2srLwuADGzJdpszWbeZsgh,77S5QQdlLJmr3HT8gMuWS9,63clD8ke57qeOw7czBPvoz,7HtiS6ubdQ8vUOMDLltiT3,4x9nVGF0fDaffc1LkBmgNP,05cjQouqyDYG6F2NSNJL7E,4F76PwZIzy2gW0wcGQuzN6,6iewIywVK4V0SQfSCZonCR,5mQlRdP91RtczegYXG3X63,4UoYb2HJklHzXbVA5klJGt,17KR2xwfXqLDwfibhi7zqZ,59OZiGqFfWmry6o9pTx8i8,3WiissyVFRpCVrqbyPKZxZ,4BYGxv4rxSNcTgT3DsFB9o,3jgZ09cCEn3udRr7KV86oJ,2S3qN8s6dS8QR5PthnqI2P,2S3qN8s6dS8QR5PthnqI2P,2jEfrzuEVCkNLbCe3EI5zY,7HT6TuyQsBNGghhYYzInsU,2XwYeBzumtjxVbFWwhJpsy,6Ktr87bevDceLjUC3xtLAt,6G9CeRGryVV0ZX60fkMEeO,6TDGueJuplnLpbww2YQPM0,7HAliS4AAbY8kWnhotOyTx,2Ajr7w89EnxLZ6KONUAguF,3o5ibU6l0GRUzzg8Hlk6xJ,7lMjqRbsChqroWRRkhw3ge,3Rv7s6zTx6KTYZfrpBdW4C,0Qj2eDY1Nku7OGymMs8HGz,2pgLRrQEvDwd6jTXGDswQm,5SzbfdSfxmmlyTEZcpb3EH,6wUEUO6bE1R84LAB7TIAIs,09u4feGdXwYsvH7hWObLPw,09u4feGdXwYsvH7hWObLPw,0q4ADFP0VAFpTKyAnWuU0e,7mxdqQjhmmv1iPdrE6VEmB
track_idi,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
57lYOdic1C2GVyTPZsJhlB,0.000000,1.558289,1.377439,1.347754,1.297031,1.137113,1.409339,1.017711,1.017711,0.880837,1.003425,0.414999,0.509516,1.240029,1.089821,0.718603,1.449951,1.022363,0.848759,0.779702,1.008442,0.941454,1.340371,1.110847,1.341416,0.511331,1.017996,1.017996,0.664518,1.369474,1.413098,1.342926,0.750395,1.270100,1.318388,1.072619,0.954568,1.027653,0.674461,1.206195,...,1.057383,0.732284,1.411615,1.059114,1.242093,1.399485,1.055869,0.599201,0.600424,1.231498,0.787953,0.848354,1.053290,1.106828,1.478660,1.143241,0.638446,0.912801,1.056659,0.434468,0.434468,0.565150,1.046180,1.440881,1.054054,0.581304,0.922916,0.819850,1.371879,0.418161,0.977466,0.235986,0.448948,0.353376,0.301041,0.652986,6.653342e-02,6.653342e-02,0.091819,0.572268
09u4feGdXwYsvH7hWObLPw,0.066533,1.475938,1.515658,1.221611,1.241015,1.288797,1.305671,1.151899,1.151899,0.819873,0.924658,0.395024,0.566905,1.055697,1.043086,0.760326,1.416039,0.884601,0.734737,0.739803,1.152737,0.939717,1.334393,1.133570,1.327227,0.705882,0.896885,0.896885,0.691440,1.532996,1.497277,1.513010,0.710954,1.407695,1.256255,1.078675,0.933556,0.980742,0.651732,1.107365,...,0.991088,0.635505,1.351285,0.911422,1.199091,1.450328,1.027098,0.544185,0.473770,1.427946,0.649374,0.751898,1.013175,0.972594,1.613967,1.195810,0.541773,0.919952,1.105237,0.386113,0.386113,0.556673,0.991433,1.559777,0.954739,0.631402,1.106093,0.881460,1.204815,0.480370,0.974831,0.272729,0.414953,0.254431,0.549292,0.597994,3.330669e-16,0.000000e+00,0.086849,0.532806
09u4feGdXwYsvH7hWObLPw,0.066533,1.475938,1.515658,1.221611,1.241015,1.288797,1.305671,1.151899,1.151899,0.819873,0.924658,0.395024,0.566905,1.055697,1.043086,0.760326,1.416039,0.884601,0.734737,0.739803,1.152737,0.939717,1.334393,1.133570,1.327227,0.705882,0.896885,0.896885,0.691440,1.532996,1.497277,1.513010,0.710954,1.407695,1.256255,1.078675,0.933556,0.980742,0.651732,1.107365,...,0.991088,0.635505,1.351285,0.911422,1.199091,1.450328,1.027098,0.544185,0.473770,1.427946,0.649374,0.751898,1.013175,0.972594,1.613967,1.195810,0.541773,0.9199

In [ ]:
#Check what track it is by transposing the matrix.  This is cool trick.
#NOTE: The INDEX needed to be set to 'track_id' for af_rand_10_df.  A column
# assignment wont work for some reason.

def aboutTrack(df,track_id):
  track_check = df.T[track_id]
  return track_check

In [ ]:
#Use aboutTrack function

recommended_track_data = aboutTrack(af_rand_10_df,'2vSjs5QYsnVfp1HIDMhHdh')
recommended_track_data

track_id                                                   2vSjs5QYsnVfp1HIDMhHdh
track_name                        The Day is Past and Gone (Variations), Extended
album_id                                                   6AwdnAvL3eD7oH87pDtrpe
album_name                                                            Field Guide
artist_id                                                  2lGwVpVjE5WrnI4pP5Lu0y
artist_name                                                          Sarah Louise
track_preview_url                                                             NaN
track_number                                                                    5
track_href                      https://api.spotify.com/v1/tracks/2vSjs5QYsnVf...
track_external_urls             https://open.spotify.com/track/2vSjs5QYsnVfp1H...
track_external_id_isrc                                               USQY51583159
album_href                      https://api.spotify.com/v1/albums/6AwdnAvL3eD7...
album_ext_url   

# (Optional) Combining Dataframes Prior to Calculating Cosine Distances

Sometimes it is best to output your API results sequentially to a CSV so you don't lose your progress.  In this case, I recommend labelling your csv files with the range of tracks you've taken and then combine then later into one dataframe.

In the example below, I took ~the first 40,000 tracks and combined them with the ~42,000 tracks that make up the entire spotify official playlist library.

In [ ]:
#In this example 'af' = 'audio features', 'tot' = 'total', and the numbers correspond to the
# thousands of tracks.

af_tot_df_0_to_40 = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df_0_40.csv')
af_tot_df_0_to_40 = af_tot_df_0_to_40.drop(columns=['Unnamed: 0','index'])
af_tot_df_40_to_80 = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df_40_to_80.csv')
af_tot_df_40_to_80 = af_tot_df_40_to_80.drop(columns=['Unnamed: 0','index'])
af_tot_80_df = pd.concat((af_tot_df_0_to_40,af_tot_df_40_to_80), axis=0)

In [ ]:
'''#Import the csv.  Make sure to add the directory if you aren't importing
# from the current directory.  You can find this by opening your terminal
# (on Mac) and dragging the csv into the terminal.  The full address will
# be displayed on the command line.


#Drop unnessesary columns
#Set 'track_id' as the index

af_tot_df_0_40 = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df_0_40.csv')
af_tot_df_0_40 = af_tot_df_0_40.drop(columns=['Unnamed: 0','index'])
af_tot_df_0_40'''

In [ ]:
'''#This dataframe was made from 2 dataframes ~40,000 tracks each.
#I finished doing a single export later.  The line for reading that in
# is af_tot_df = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_80_df.csv')

#Use this cell instead of the partial one with just af_tot_df_0_40.

af_tot_df_0_to_40 = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df_0_40.csv')
af_tot_df_0_to_40 = af_tot_df_0_to_40.drop(columns=['Unnamed: 0','index'])
af_tot_df_40_to_80 = pd.read_csv('/content/drive/MyDrive/Project5/af_tot_df_40_to_80.csv')
af_tot_df_40_to_80 = af_tot_df_40_to_80.drop(columns=['Unnamed: 0','index'])
af_tot_df = pd.concat((af_tot_df_0_to_40,af_tot_df_40_to_80), axis=0)
af_tot_df.to_csv('/content/drive/MyDrive/Project5/af_tot_df.csv')'''

# Column Manipulation references

In [ ]:
#rename col 'track_id' to 'track_idi" for "track id index" for clarity
af_tot_df.rename(columns={'track_id':'track_idi'},inplace=True)

In [ ]:
#move 'track_id2' to column=0 by doing a list comprehension where the dataframe = the column you want to move plus every column that isn't equal to that column.
af_tot_df = af_tot_df[['track_id'] + [col for col in af_tot_df.columns if col != 'track_id2' ]]

# Song Tracer track_id reference

In [ ]:
vaso_vagal=['57lYOdic1C2GVyTPZsJhlB',
            '3Rv7s6zTx6KTYZfrpBdW4C',
            '0Qj2eDY1Nku7OGymMs8HGz',
            '2pgLRrQEvDwd6jTXGDswQm',
            '5SzbfdSfxmmlyTEZcpb3EH',
            '6wUEUO6bE1R84LAB7TIAIs',
            '09u4feGdXwYsvH7hWObLPw',
            '0q4ADFP0VAFpTKyAnWuU0e',
            '7mxdqQjhmmv1iPdrE6VEmB']